In [12]:
from pykrx import stock
from pykrx import bond
import pandas as pd
from datetime import datetime

In [13]:
now = datetime.now()
now = now.strftime('%Y-%m-%d')

In [14]:
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))
stock_list.index = stock_list['종목코드']

df1 = stock.get_market_ohlcv(now, market="ALL") #시세정보
df2 = stock.get_market_fundamental(now, market="ALL") #PER등 재무정보
df3 = stock.get_market_cap(now, market="ALL") #시가총액 조회

df_merged = pd.concat([df1,df2], axis=1) 
df_merged = df_merged[['시가', '고가', '저가','등락률', 'BPS', 'PER', 'PBR','EPS', 'DIV', 'DPS']]
df_merged = pd.merge(df_merged, df3, left_index=True, right_index=True,how='outer')
df_merged = pd.merge(df_merged, stock_list, left_index=True, right_index=True,how='outer')

In [15]:
df_merged['전일종가'] = (df_merged['종가'] / (df_merged['등락률']/100+1)).round()
df_merged['고가백분율'] = (((df_merged['고가']-df_merged['전일종가'])/df_merged['전일종가'])*100).round(2)
df_merged['저가백분율'] = (((df_merged['저가']-df_merged['전일종가'])/df_merged['전일종가'])*100).round(2)
df_merged = df_merged.fillna('-')
df_merged = df_merged[['종목명','거래량','등락률','고가백분율','저가백분율','거래대금','시가총액','시가','고가',\
                       '저가','BPS','PER','PBR','EPS','DIV','DPS','종가','상장주식수','종목코드','전일종가']]
df_merged.to_excel("test_daily_result.xlsx")

In [23]:
df_sd = pd.read_excel('c:/temp/시그널다이제스트.xlsx') #시그널다이제스트를 에버노트로 변환해서 전환한뒤 다이제스트부분만 오려서 엑셀에 붙여놓고 시그널다이제스트.xlsx로 저장함.
df_sd = df_sd.dropna()
df_sd = df_sd.reset_index(drop=True)
df_sd.columns = ['content']

df_sd_stock = df_sd[df_sd['content'].str.contains('관련주: ')]
stock_series = df_sd_stock.content.str.split('관련주: ').str[1]
stock_series = stock_series.reset_index()['content']
stock_df = pd.DataFrame(stock_series)
stock_df['list'] = stock_series.str.split(', ')
stock_df['no'] = stock_df.index #일단 인덱스로 저장하고, 나중에 구문 분석이 완성되면 해당 기사와 리포트의 날짜를 같이 넣어서 사용

In [25]:
no = 0
list_df = pd.DataFrame()
for list in stock_df['list']:
    tmp_df = pd.DataFrame(list)
    tmp_df['no'] = str(no)
    list_df = pd.concat([list_df,tmp_df],ignore_index=True)
    no += 1
list_df.columns = ['종목명','기사no']

In [18]:
final_df = pd.merge(list_df,df_merged)

In [19]:
final_df.to_excel('시그널다이제스트결과.xlsx')